In [1]:
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model



In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # load environment variables from .env

# --- Environment Variable Checks ---
google_api_key_loaded = os.getenv("GOOGLE_API_KEY") is not None

print(f"Google API Key Loaded: {google_api_key_loaded}")



Google API Key Loaded: True


In [3]:
model = init_chat_model("gemini-2.0-flash", 
                        model_provider="google_genai",
                        temperature=0)
# model = init_chat_model("gemini-1.5-flash", model_provider="google_genai")

response = model.invoke("Hello, world!")
print(response.content)

Hello there! How can I help you today?


In [4]:
from langgraph.prebuilt import create_react_agent

car_agent = create_react_agent(
    model=model,
    tools=[],
    prompt=(
        "You are a car agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with research-related to cars, DO NOT provide any other category related data\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="car_agent",
)

In [5]:
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

In [6]:
for chunk in car_agent.stream(
    {"messages": [{"role": "user", "content": "what are 2 indian cars?"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: car_agent

- Tata Nexon
- Mahindra XUV700




### Math Agent

In [7]:
def add(a: float, b: float):
    """Add two numbers."""
    return a + b


def multiply(a: float, b: float):
    """Multiply two numbers."""
    return a * b


def divide(a: float, b: float):
    """Divide two numbers."""
    return a / b


math_agent = create_react_agent(
    model=model,
    tools=[add, multiply, divide],
    prompt=(
        "You are a math agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with math-related tasks\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="math_agent",
)

In [8]:
for chunk in math_agent.stream(
    {"messages": [{"role": "user", "content": "what's (3+5)*7"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: math_agent
Tool Calls:
  add (d351ab06-d29e-4215-a606-9bfe635382aa)
 Call ID: d351ab06-d29e-4215-a606-9bfe635382aa
  Args:
    b: 5.0
    a: 3.0


Update from node tools:


================================= Tool Message =================================
Name: add

8.0


Update from node agent:


================================== Ai Message ==================================
Name: math_agent
Tool Calls:
  multiply (f0c76bc8-2b64-48c6-8014-03566a7af14b)
 Call ID: f0c76bc8-2b64-48c6-8014-03566a7af14b
  Args:
    b: 7.0
    a: 8.0


Update from node tools:


================================= Tool Message =================================
Name: multiply

56.0


Update from node agent:


================================== Ai Message ==================================
Name: math_agent

56




## Create supervisor with langgraph-supervisor

In [72]:
from langgraph_supervisor import create_supervisor
from langchain.chat_models import init_chat_model

supervisor = create_supervisor(
    model=model,
    agents=[car_agent, math_agent],
    prompt=(
        "You are a supervisor managing two agents:\n"
        "- a car agent. Assign car-related tasks to this agent\n"
        "- a math agent. Assign math-related tasks to this agent\n"
        "Assign work to one agent at a time, do not call agents in parallel.\n"
        "Do not do any work yourself."
    ),
    add_handoff_back_messages=True,
    output_mode="full_history",
).compile()

In [ ]:
from IPython.display import display, Image

display(Image(supervisor.get_graph().draw_mermaid_png()))

In [ ]:
# Assuming 'supervisor' is your compiled LangGraph StateGraph or CompiledGraph
mermaid_syntax = supervisor.get_graph().draw_mermaid()
print(mermaid_syntax)

# Example of how you might use it in a Jupyter Notebook markdown cell:
# ```mermaid
# # Paste the output of print(mermaid_syntax) here
# ```


---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	supervisor(supervisor)
	car_agent(car_agent)
	math_agent(math_agent)
	__end__([<p>__end__</p>]):::last
	__start__ --> supervisor;
	car_agent --> supervisor;
	math_agent --> supervisor;
	supervisor -.-> __end__;
	supervisor -.-> car_agent;
	supervisor -.-> math_agent;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "I want to buy tata tiago car in hyderabad. What is the EMI i need to pay for a downpayment of 5 Lakh rupees and a 10% intrest rate for 3 years. The car costs 10Lakh rupeers and the loan is 5 Lakhs.",
            }
        ]
    },
):x
    pretty_print_messages(chunk, last_message=True)

final_message_history = chunk["supervisor"]["messages"]

Update from node supervisor:


================================= Tool Message =================================
Name: transfer_to_math_agent

Successfully transferred to math_agent


Update from node math_agent:


================================= Tool Message =================================
Name: transfer_back_to_supervisor

Successfully transferred back to supervisor


Update from node supervisor:


================================== Ai Message ==================================
Name: supervisor

OK. The EMI you need to pay is 16,187.76 rupees.


